# Case Final
## Análise de Transações PIX
CRISP-DM - https://www.escoladnc.com.br/blog/data-science/metodologia-crisp-dm/
### Objetivos
- Limpar e pré-processar os dados das transações PIX
- Analisar padrões de uso do PIX, tais como os canais mais utilizados e os valores de transação mais comuns
- Use o PySpark MLlib para treinar e avaliar um modelo de detecção de fraude
- Avaliar o desempenho do modelo e fazer recomendações para melhorias futuras

### Dados

O conjunto de dados inclui as seguintes informações para cada transação:
- Detalhes da transação: valor, tempo, remetente e receptor CPF/CNPJ, tipo
- Etiqueta de fraude: uma variável binária que indica se a transação foi fraudulenta (1) ou não (0)

### Tarefas
- Normalização dos dados:
  - O dataset que você lerá está em formato json.
  ```json
  {
      "id_transacao": inteiro,
      "valor": texto,
      "remetente": {
          "nome": texto,
          "banco": texto,
          "tipo": texto
      },
      "destinatario": {
          "nome": texto,
          "banco":texto,
          "tipo": texto
      },          
      "categoria":texto,
      "chave_pix":texto,
      "transaction_date":texto,
      "fraude":inteiro,
  }
    ```
  - Faça sua transformação para formato colunar
- Análise Exploratória de Dados: Use o PySpark para analisar padrões de uso do PIX:
  - chaves pix mais usadas;
  - os valores de transação mais comuns;
  - distribuição dos valores de transação por hora e dia;
  - quais bancos receberam mais transferências por dia;
  - para qual tipo de pessoa (PF ou PJ) foram realizadas mais transações
- Engenharia de Recursos: Apresentar novas características que podem ser úteis para a detecção de fraudes, tais como o número de transações feitas pelo mesmo remetente em um período de tempo específico.
- Modelagem: Use o PySpark MLlib para treinar e detectar possíveis transações que contenham fraude.

### Observação
É importante notar que este é um caso simplificado, e em cenários do mundo real você teria que lidar com dados mais complexos, usar técnicas mais avançadas como métodos de conjuntos e considerar o conhecimento de domínio, bem como leis e regulamentos das instituições financeiras no Brasil.


### Observação II
Não existe resposta 100% correta. É necessário que você use seu pensamento crítico para definir as melhores métricas e análises para o caso.

# Entendimento do Negócio
Você trabalha em um banco e o principal meio de pagamento utilizado no seu banco é o Pix.

Através da base de transações do pix o banco deseja entender qual é o perfil dos clientes que utilizam o pix, além de verificar possíveis transações que tenham fraude. Porém, eles tem um cliente específico que tem um relacionamento muito bom para o banco, por isso, você recebeu a base de transações de cliente dos últimos 2 anos e precisa a partir dela criar um relatório contendo as principais características das transações.


Então, resumindo, temos dois principais objetivos para esse case:
1. Obter valor a partir dos dados
  - Para qual banco esse cliente mais transfere?
  - Qual é a média de transferências por período que esse cliente faz?
  - Baseando-se no valor das transferências, poderia dar um aumento de crédito?
  - Para o que esse cliente mais usa as transferências?
2. Executar um algoritmo de machine learning que identifique possíveis transações com fraude.
3. Pós Processamento
  - Defina ao mínimo cinco métricas de qualidade para seus dados
  - Explique se os seus dados estão com uma boa qualidade

# Preparação do Ambiente de Desenvolvimento

# Data Undesrtanting

Primeiramente, devemos entender tudo sobre a fonte dos dados
- Como o dado chega até nós?
- Qual formato virá?
- Aonde o processamento será executado (AWS EMR, Cluster On-Premise)?
- De quanto em quanto tempo eu preciso gerar esse relatório (mensal, diário, near-real time)?

```json
{
  "id_transacao": inteiro,
  "valor": texto,
  "remetente": {
      "nome": texto,
      "banco": texto,
      "tipo": texto
  },
  "destinatario": {
      "nome": texto,
      "banco":texto,
      "tipo": texto
  },
  "categoria": texto,
  "transaction_date":texto,
  "chave_pix":texto,
  "fraude":inteiro,
}
```



# Preparação do Ambiente de Desenvolvimento

In [1]:
# Instalar a última versão do PySpark
!pip install pyspark #==3.3.1

# Instalar o NGROK
!wget -qnc https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -n -q ngrok-stable-linux-amd64.zip


# Iniciar a sessão spark
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
      .config('spark.ui.port', '4050')
      .appName("CaseFinal")
      .getOrCreate()
)

# Autenticar a sessão do SparkUI com NGROK
!./ngrok authtoken 2KBeQEmmd1YNlQ86GGKf3KFOkb3_6sQH7JEnvEhDxwn9A7WnT
get_ipython().system_raw('./ngrok http 4050 &')
!sleep 10
!curl -s http://localhost:4040/api/tunnels | grep -Po 'public_url":"(?=https)\K[^"]*'

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Preparação dos Dados
Agora é hora de começar a preparar os dados de acordo com as suas necessidades.

In [3]:
from pyspark.sql.types import *

# Schema para o campo rementente e destinatário
schema_remetente_destinatario = StructType([
    StructField('nome', StringType()),
    StructField('banco', StringType()),
    StructField('tipo', StringType())
])

# Schema para o restante da base
schema_base_pix = StructType([
    StructField('id_transacao', IntegerType()),
    StructField('valor', DoubleType()),
    StructField('remetente', schema_remetente_destinatario),
    StructField('destinatario', schema_remetente_destinatario),
    StructField('chave_pix', StringType()),
    StructField('categoria', StringType()),
    StructField('transaction_date', StringType()),
    StructField('fraude', IntegerType())
])

caminho_json = '/content/drive/MyDrive/Colab Notebooks/DNC - Ciencia de Dados/SparkML/case_final.json'

df = spark.read.json(
    caminho_json,
    schema=schema_base_pix,
    # Formato da hora: 2022-02-25 09:31:47 (verificado dentro do arquivo .json)
    timestampFormat="yyyy-MM-dd HH:mm:ss"
)
df.printSchema()
df.show()

root
 |-- id_transacao: integer (nullable = true)
 |-- valor: double (nullable = true)
 |-- remetente: struct (nullable = true)
 |    |-- nome: string (nullable = true)
 |    |-- banco: string (nullable = true)
 |    |-- tipo: string (nullable = true)
 |-- destinatario: struct (nullable = true)
 |    |-- nome: string (nullable = true)
 |    |-- banco: string (nullable = true)
 |    |-- tipo: string (nullable = true)
 |-- chave_pix: string (nullable = true)
 |-- categoria: string (nullable = true)
 |-- transaction_date: string (nullable = true)
 |-- fraude: integer (nullable = true)

+------------+------------------+--------------------+--------------------+---------+-------------+-------------------+------+
|id_transacao|             valor|           remetente|        destinatario|chave_pix|    categoria|   transaction_date|fraude|
+------------+------------------+--------------------+--------------------+---------+-------------+-------------------+------+
|        1000|            588

In [4]:
from pyspark.sql.functions import *
# Cria um novo dataframe com as colunas achatadadas
df_flatten = df.withColumns({
    # Extrai o campo 'nome' de dentro da estrutura 'destinatario'
    'destinatario_nome': col('destinatario').getField('nome'),
    # Extrai o campo 'banco' de dentro da estrutura 'destinatario'
    'destinatario_banco': col('destinatario').getField('banco'),
    # Extrai o campo 'tipo' de dentro da estrutura 'destinatario'
    'destinatario_tipo': col('destinatario').getField('tipo'),
    # Extrai o campo 'nome' de dentro da estrutura 'remetente'
    'remetente_nome': col('remetente').getField('nome'),
    # Extrai o campo 'banco' de dentro da estrutura 'remetente'
    'remetente_banco': col('remetente').getField('banco'),
    # Extrai o campo 'tipo' de dentro da estrutura 'remetente'
    'remetente_tipo': col('remetente').getField('tipo'),
}).drop('remetente', 'destinatario') # Dropa do df as estruturas remente e destinatario

df_flatten.show()

+------------+------------------+---------+-------------+-------------------+------+--------------------+------------------+-----------------+------------------+---------------+--------------+
|id_transacao|             valor|chave_pix|    categoria|   transaction_date|fraude|   destinatario_nome|destinatario_banco|destinatario_tipo|    remetente_nome|remetente_banco|remetente_tipo|
+------------+------------------+---------+-------------+-------------------+------+--------------------+------------------+-----------------+------------------+---------------+--------------+
|        1000|            588.08|aleatoria|       outros|2021-07-16 05:00:55|     0|         Calebe Melo|             Caixa|               PF|Jonathan Gonsalves|            BTG|            PF|
|        1001|           80682.5|  celular|transferencia|2022-04-20 12:34:01|     1|  Davi Lucas Pereira|             Caixa|               PJ|Jonathan Gonsalves|            BTG|            PF|
|        1002|             549.9|  

In [5]:
# Estatísticas do df_flatten
df_flatten.describe().show()

+-------+-----------------+------------------+---------+-----------+-------------------+------------------+-----------------+------------------+-----------------+------------------+---------------+--------------+
|summary|     id_transacao|             valor|chave_pix|  categoria|   transaction_date|            fraude|destinatario_nome|destinatario_banco|destinatario_tipo|    remetente_nome|remetente_banco|remetente_tipo|
+-------+-----------------+------------------+---------+-----------+-------------------+------------------+-----------------+------------------+-----------------+------------------+---------------+--------------+
|  count|           100000|            100000|   100000|     100000|             100000|            100000|           100000|            100000|           100000|            100000|         100000|        100000|
|   mean|          50999.5|10303.358732200059|     NULL|       NULL|               NULL|           0.15367|             NULL|              NULL|    

# Modelagem
Aqui você encontrará utilidade para os dados levantados.

Aqui será onde teremos insights e, a partir desses, novos conhecimentos sobre o business (se tudo até aqui foi feito corretamente).


- Para qual banco esse cliente mais transfere?
- Qual é a média de transferências por período que esse cliente faz?
- Baseando-se no valor das transferências, poderia dar um aumento de crédito?
- Para o que esse cliente mais usa as transferências?
- Executar um algoritmo de machine learning que identifique possíveis transações com fraude.


In [13]:
# Agrupando por destinatario_banco e contando as transações
df_flatten.groupBy('destinatario_banco').count().orderBy('count').show()

+------------------+-----+
|destinatario_banco|count|
+------------------+-----+
|          Bradesco|14187|
|                C6|14204|
|             Caixa|14240|
|              Itau|14281|
|            Nubank|14297|
|               BTG|14390|
|                XP|14401|
+------------------+-----+



In [16]:
# Verificando o total de transações por mês para cada banco
df_flatten.groupBy(
    date_format(col('transaction_date'), 'yyyy-MM').alias('ano_mes'),
    'destinatario_banco'
).count().orderBy(col('ano_mes').desc()).show()

+-------+------------------+-----+
|ano_mes|destinatario_banco|count|
+-------+------------------+-----+
|2023-01|              Itau|  267|
|2023-01|             Caixa|  277|
|2023-01|                XP|  277|
|2023-01|          Bradesco|  280|
|2023-01|            Nubank|  290|
|2023-01|                C6|  290|
|2023-01|               BTG|  278|
|2022-12|                XP|  615|
|2022-12|               BTG|  603|
|2022-12|                C6|  576|
|2022-12|          Bradesco|  575|
|2022-12|            Nubank|  602|
|2022-12|              Itau|  633|
|2022-12|             Caixa|  616|
|2022-11|          Bradesco|  579|
|2022-11|               BTG|  580|
|2022-11|              Itau|  614|
|2022-11|            Nubank|  620|
|2022-11|             Caixa|  543|
|2022-11|                C6|  561|
+-------+------------------+-----+
only showing top 20 rows



In [19]:
# Valor médio das transações por banco
df_flatten.groupBy('destinatario_banco').avg('valor').orderBy('avg(valor)').show()

+------------------+------------------+
|destinatario_banco|        avg(valor)|
+------------------+------------------+
|               BTG|10122.299803335622|
|              Itau|10230.876305580874|
|             Caixa|10254.864015449395|
|                C6|10309.499774711307|
|            Nubank|10316.475401133126|
|                XP|10328.071572113045|
|          Bradesco| 10564.19458870794|
+------------------+------------------+



In [21]:
# Categoria por cada banco
df_flatten.groupBy(date_format(col('transaction_date'), 'yyyy-MM').alias('ano_mes'),
                   'destinatario_banco', 'categoria').count().orderBy('ano_mes').show()

+-------+------------------+-------------+-----+
|ano_mes|destinatario_banco|    categoria|count|
+-------+------------------+-------------+-----+
|2021-01|             Caixa|     educacao|   24|
|2021-01|              Itau|        saude|   40|
|2021-01|            Nubank|        saude|   28|
|2021-01|                XP|    presentes|   38|
|2021-01|               BTG|   transporte|   29|
|2021-01|               BTG|  alimentacao|   31|
|2021-01|              Itau|transferencia|   75|
|2021-01|                XP|       outros|   34|
|2021-01|                XP|   transporte|   26|
|2021-01|          Bradesco|transferencia|   96|
|2021-01|              Itau|    vestuario|   28|
|2021-01|               BTG|        lazer|   34|
|2021-01|          Bradesco|   transporte|   40|
|2021-01|                XP|transferencia|   89|
|2021-01|                C6|  alimentacao|   38|
|2021-01|             Caixa|        lazer|   37|
|2021-01|          Bradesco|       outros|   25|
|2021-01|           

In [29]:
# Soma o quanto o usuário gasta por categoria por ano
df_flatten.groupBy(
    date_format(col('transaction_date'), 'yyyy').alias('ano'),
    'categoria'
).sum('valor').select('ano', 'categoria', col('sum(valor)').cast(DecimalType(38, 3)).alias('valor')).orderBy('valor').show(30)

+----+-------------+-------------+
| ano|    categoria|        valor|
+----+-------------+-------------+
|2023|    presentes|   362584.450|
|2023|  alimentacao|   392078.310|
|2023|        saude|   400683.640|
|2023|   transporte|   427790.540|
|2023|     educacao|   432305.660|
|2023|    vestuario|   459528.700|
|2023|        lazer|   469671.410|
|2023|       outros|   501308.340|
|2021|   transporte|  9497893.470|
|2021|        lazer|  9622503.140|
|2021|       outros|  9737076.380|
|2022|        saude| 10048245.070|
|2021|     educacao| 10106095.070|
|2021|  alimentacao| 10237928.250|
|2022|        lazer| 10295747.630|
|2022|    presentes| 10311585.740|
|2022|     educacao| 10367124.440|
|2021|        saude| 10384662.820|
|2021|    vestuario| 10412651.930|
|2021|    presentes| 10496042.020|
|2022|  alimentacao| 10545783.230|
|2022|   transporte| 10553408.290|
|2022|       outros| 10566645.100|
|2022|    vestuario| 10696006.480|
|2023|transferencia| 16148682.620|
|2021|transferencia|

In [30]:
# Média de transações po ano
df_flatten.groupBy(
    date_format(col('transaction_date'), 'yyyy').alias('ano')
).avg('id_transacao').select('ano',col('avg(id_transacao)').alias('avg')).orderBy('ano').show(30)

+----+------------------+
| ano|               avg|
+----+------------------+
|2021| 51204.03597092333|
|2022|50823.494263441935|
|2023| 50481.67993874426|
+----+------------------+



In [31]:
# Quantidade de transações fraudes e não fraudes
df_flatten.groupBy('fraude').count().show()

+------+-----+
|fraude|count|
+------+-----+
|     1|15367|
|     0|84633|
+------+-----+



In [33]:
# Verificando quais categorias tem fraudes
df_flatten.groupBy('categoria','fraude').count().show()

+-------------+------+-----+
|    categoria|fraude|count|
+-------------+------+-----+
|       outros|     0| 9377|
|     educacao|     0| 9460|
|transferencia|     0| 9377|
|transferencia|     1|15367|
|    presentes|     0| 9254|
|        saude|     0| 9476|
|        lazer|     0| 9464|
|   transporte|     0| 9174|
|    vestuario|     0| 9503|
|  alimentacao|     0| 9548|
+-------------+------+-----+



In [34]:
# Quantidade de fraude e não fraude apenas em transferencia
df_flatten.filter(
    col('categoria') == 'transferencia'
).groupBy('categoria','fraude').count().show()

+-------------+------+-----+
|    categoria|fraude|count|
+-------------+------+-----+
|transferencia|     0| 9377|
|transferencia|     1|15367|
+-------------+------+-----+



In [35]:
df_flatten.filter(col('fraude') == 1).withColumn(
    "range",
    floor(col("valor")/1000)*1000 # Cria um range de 1000 em 1000
).groupBy('range').count().orderBy(col('range').desc()).show()

+-----+-----+
|range|count|
+-----+-----+
|89000|  222|
|88000|  208|
|87000|  230|
|86000|  203|
|85000|  205|
|84000|  245|
|83000|  206|
|82000|  206|
|81000|  214|
|80000|  213|
|79000|  205|
|78000|  230|
|77000|  237|
|76000|  232|
|75000|  190|
|74000|  207|
|73000|  237|
|72000|  234|
|71000|  234|
|70000|  222|
+-----+-----+
only showing top 20 rows



In [37]:
# transferencia minima e máxima com fraude
df_flatten.filter(col('fraude') == 1).withColumn(
    "range",
    floor(col("valor")/1000)*1000 # Cria um range de 1000 em 1000
).select(max('range'), min('range')).show()

+----------+----------+
|max(range)|min(range)|
+----------+----------+
|     89000|     19000|
+----------+----------+



## Modelo de Predição de Fraudes

In [59]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorIndexer, VectorAssembler
from pyspark.ml.classification import LogisticRegression

In [39]:
df_flatten.drop('remetente','id_transacao')

DataFrame[valor: double, chave_pix: string, categoria: string, transaction_date: string, fraude: int, destinatario_nome: string, destinatario_banco: string, destinatario_tipo: string, remetente_nome: string, remetente_banco: string, remetente_tipo: string]

In [41]:
indexer = StringIndexer(
    # Lista de colunas categóricas
    inputCols=[
        'destinatario_nome',
        'destinatario_banco',
        'destinatario_tipo',
        'categoria',
        'chave_pix'
    ],
    # Nome das colunas transformadas em numéricas
    outputCols=[
        'destinatario_nome_index',
        'destinatario_banco_index',
        'destinatario_tipo_index',
        'categoria_index',
        'chave_pix_index'
    ]
)

# Novo dataframe que recebera as colunas transformadas
df_index = indexer.fit(df_flatten).transform(df_flatten)
df_index.show()

+------------+------------------+---------+-------------+-------------------+------+--------------------+------------------+-----------------+------------------+---------------+--------------+-----------------------+------------------------+-----------------------+---------------+---------------+
|id_transacao|             valor|chave_pix|    categoria|   transaction_date|fraude|   destinatario_nome|destinatario_banco|destinatario_tipo|    remetente_nome|remetente_banco|remetente_tipo|destinatario_nome_index|destinatario_banco_index|destinatario_tipo_index|categoria_index|chave_pix_index|
+------------+------------------+---------+-------------+-------------------+------+--------------------+------------------+-----------------+------------------+---------------+--------------+-----------------------+------------------------+-----------------------+---------------+---------------+
|        1000|            588.08|aleatoria|       outros|2021-07-16 05:00:55|     0|         Calebe Melo| 

In [42]:
# Filtro para separar as colunas necessárias
cols_para_filtrar = [
  "valor",
  "transaction_date",
  "destinatario_nome_index",
  "destinatario_banco_index",
  "destinatario_tipo_index",
  "chave_pix_index",
  "categoria_index",
  "fraude"
]

In [49]:
# Filtro com apenas transações com fraude
is_fraud = df_index.select(cols_para_filtrar).filter('fraude==1')
# Filtro com apenas transações sem fraude
no_fraud = df_index.select(cols_para_filtrar).filter('fraude==0')
# Amostra de 1% dos dados que não são fraudes
no_fraud = no_fraud.sample(False, 0.01, seed=42)

In [51]:
# Total de fraudes
is_fraud.count()

15367

In [52]:
# Total sem fraude
no_fraud.count()

902

In [50]:
# Concatenando os df que não contém fraude com o que tem fraude
df_concat = no_fraud.union(is_fraud)
df = df_concat.sort('transaction_date')
df.count()

16269

In [53]:
# Separando em treino(70%) e teste (30%)
train, test = df.randomSplit([0.7, 0.3], seed = 42)
print("train =", train.count(), " test =", test.count())

train = 11491  test = 4778


In [56]:
# Se o valor for maior que 0, retorna 1.0 (indica fraude); caso contrário, retorna 0.0 (sem fraude)
is_fraud = udf(lambda fraud: 1.0 if fraud > 0 else 0.0, DoubleType())
train = train.withColumn("is_fraud", is_fraud(train.fraude))

In [60]:
# Cria o vetor assembler (transfoma as culunas transaction_date, fraude e is_fraud numa coluna só)
assembler = VectorAssembler(
  inputCols = [x for x in train.columns if x not in ["transaction_date", "fraude", "is_fraud"]],
  outputCol = "features")

# Regressão Logística
lr = LogisticRegression().setParams(
    maxIter = 100000,
    labelCol = "is_fraud",
    predictionCol = "prediction")


# Pipeline com o assembler e a regressão logística
model = Pipeline(stages = [assembler, lr]).fit(train)

In [61]:
predicted = model.transform(test)

predicted = predicted.withColumn("is_fraud", is_fraud(predicted.fraude))
predicted.crosstab("is_fraud", "prediction").show()

+-------------------+---+----+
|is_fraud_prediction|0.0| 1.0|
+-------------------+---+----+
|                1.0|  0|4544|
|                0.0|233|   1|
+-------------------+---+----+



# Deployment
Identificado que existe um único cliente na base de dados, Jonathan Gonsalve. O mesmo tem uma alta taxa de transaferências pix mensal. Através de análise realizadas foi possível perceber que a maior categoria de transação é a transferência bancária.

Também foi possível notar que o segundo banco que o cliente mais transaciona é o banco BTG, porém, o mesmo possui o menor valor transacionado, o que indica que o cliente faz muitas transações de menor valor para esse banco.

Também foi possível verificar que há um alto índice de tentativas de fraude na conta desse cliente, sendo que todas as tentativas de fraude foram com valores acima de R$19.999,00 e com a categoria de transferência.

Conclui-se que há uma alta tentativa de transações com fraude e uma ação possível seria diminuir o limite máximo de transferência de pix do cliente.Possivelmente o cliente esteja usando essa conta PF com propósitos de PJ, devido a alta taxa de transferências com altos valores.  





  